# [Module 4.2.1] Inference Custom Docker Image 생성 및 ECR 퍼블리시

이 노트북은 Docker 이미지를 생성하고, Amazon ECR(Elastic Container Registry)에 퍼블리스를 합니다.
SageMaker 는 Custom Tensorflow Serving 이미지를 만들 수 있게 필요한 리소스(Dockerfile등)을 제공 합니다.

아래와 같은 작업을 진행 합니다.
- [SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container) 을 Git Clone을 하여 다운로드 (**이 과정은 이미 다운로드 하여 현재의 Git 복사가 되어 있어서 생략 합니다.**)
- 주어지는 여러가지 버전의 Dockerfile 중에서 2.0-cpu 를 사용
    - 다른 버전을 사용하셔도 됩니다.(예: 2.0-gpu)
- 필요한 Package인 tensorflow:2.1.0 과 transformers:2.8.0을 추가
- docker image를 빌드
- ECR에 퍼블리시


---
노트북의 소요 시간은 약 10분 걸립니다.


## SageMaker TensorFlow Serving Container 다운로드
**이 부분은 이미 다운로드 했기에 진행하지 않습니다.**

* https://github.com/aws/sagemaker-tensorflow-serving-container
```
# ! git clone https://github.com/aws/sagemaker-tensorflow-serving-container
```

## Dockerfile 수정 (tfs:2.0-cpu)
다운로드 받은 폴더에서 아래 파일을 수정 하겠습니다.
- sagemaker-tensorflow-serving-container/docker/2.0/Dockerfile.cpu
- Dockerfile 안에 아래를 추가 하겠습니다.
```
#################
# 추가 사항
# Install tensorflow 2.1 and transformer
#################
RUN ${PIP} install --no-cache-dir \
    tensorflow==2.1.0 \
    transformers==2.8.0
#################
```

In [10]:
%%writefile sagemaker-tensorflow-serving-container/docker/2.0/Dockerfile.cpu 

FROM ubuntu:18.04

LABEL maintainer="Amazon AI"
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

ARG PYTHON=python3
ARG PIP=pip3
ARG TFS_SHORT_VERSION=2.0.1
ARG TFS_URL=https://tensorflow-aws.s3-us-west-2.amazonaws.com/${TFS_SHORT_VERSION}/Serving/CPU-WITH-MKL/tensorflow_model_server

# See http://bugs.python.org/issue19846
ENV LANG=C.UTF-8
# Python won’t try to write .pyc or .pyo files on the import of source modules
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV SAGEMAKER_TFS_VERSION="${TFS_SHORT_VERSION}"
ENV PATH="$PATH:/sagemaker"
ENV LD_LIBRARY_PATH='/usr/local/lib:$LD_LIBRARY_PATH'
ENV MODEL_BASE_PATH=/models
# The only required piece is the model name in order to differentiate endpoints
ENV MODEL_NAME=model
ENV DEBIAN_FRONTEND=noninteractive

# nginx + njs
RUN apt-get update \
 && apt-get -y install --no-install-recommends \
    curl \
    gnupg2 \
    ca-certificates \
    git \
    wget \
    vim \
    build-essential \
    zlib1g-dev \
 && curl -s http://nginx.org/keys/nginx_signing.key | apt-key add - \
 && echo 'deb http://nginx.org/packages/ubuntu/ bionic nginx' >> /etc/apt/sources.list \
 && apt-get update \
 && apt-get -y install --no-install-recommends \
    nginx \
    nginx-module-njs \
    python3 \
    python3-pip \
    python3-setuptools \
 && apt-get clean \
 && rm -rf /var/lib/apt/lists/*

RUN ${PIP} --no-cache-dir install --upgrade pip setuptools

# cython, falcon, gunicorn, grpc
RUN ${PIP} install --no-cache-dir \
    awscli==1.16.303 \
    cython==0.29.14 \
    falcon==2.0.0 \
    gunicorn==20.0.4 \
    gevent==1.4.0 \
    requests==2.22.0 \
    grpcio==1.26.0 \
    protobuf==3.11.1 \
# using --no-dependencies to avoid installing tensorflow binary
 && ${PIP} install --no-dependencies --no-cache-dir \
    tensorflow-serving-api==2.0

#################
# 추가 사항
# Install tensorflow 2.1 and transformer
#################
RUN ${PIP} install --no-cache-dir \
    tensorflow==2.1.0 \
    transformers==2.8.0
#################


COPY ./sagemaker /sagemaker

# Some TF tools expect a "python" binary
RUN ln -s $(which ${PYTHON}) /usr/local/bin/python

RUN curl https://tensorflow-aws.s3-us-west-2.amazonaws.com/MKL-Libraries/libiomp5.so -o /usr/local/lib/libiomp5.so
RUN curl https://tensorflow-aws.s3-us-west-2.amazonaws.com/MKL-Libraries/libmklml_intel.so -o /usr/local/lib/libmklml_intel.so

RUN curl $TFS_URL -o /usr/bin/tensorflow_model_server \
 && chmod 555 /usr/bin/tensorflow_model_server

# Expose ports
# gRPC and REST
EXPOSE 8500 8501

# Set where models should be stored in the container
RUN mkdir -p ${MODEL_BASE_PATH}

# Create a script that runs the model server so we can use environment variables
# while also passing in arguments from the docker command line
RUN echo '#!/bin/bash \n\n' > /usr/bin/tf_serving_entrypoint.sh \
 && echo '/usr/bin/tensorflow_model_server --port=8500 --rest_api_port=8501 --model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} "$@"' >> /usr/bin/tf_serving_entrypoint.sh \
 && chmod +x /usr/bin/tf_serving_entrypoint.sh

ADD https://raw.githubusercontent.com/aws/aws-deep-learning-containers-utils/master/deep_learning_container.py /usr/local/bin/deep_learning_container.py

RUN chmod +x /usr/local/bin/deep_learning_container.py

RUN curl https://aws-dlc-licenses.s3.amazonaws.com/tensorflow-2.0.1/license.txt -o /license.txt

CMD ["/usr/bin/tf_serving_entrypoint.sh"]

Overwriting sagemaker-tensorflow-serving-container/docker/2.0/Dockerfile.cpu


## Custome Docker 생성
아래 셀 스크립트를 실행하면 이미지가 생성이 됩니다. 
```
./scripts/build.sh --version 2.0.0 --arch cpu
```

In [11]:
import boto3
region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [12]:
tfs_version = "2.0.0"
cpu_type = "cpu"

In [13]:
%%bash -s {region} {tfs_version} {cpu_type}

AWS_DEFAULT_REGION=$1
export AWS_DEFAULT_REGION
TFS_VERION=$2
CPU_TYPE=$3

cd sagemaker-tensorflow-serving-container

./scripts/build.sh --version $TFS_VERION --arch $CPU_TYPE

pulling previous image for layer cache... 
building image... 
Sending build context to Docker daemon  88.58kB
Step 1/32 : FROM ubuntu:18.04
 ---> 6526a1858e5d
Step 2/32 : LABEL maintainer="Amazon AI"
 ---> Using cache
 ---> dcee4ad7f5d2
Step 3/32 : LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true
 ---> Using cache
 ---> 60840a97e0bf
Step 4/32 : ARG PYTHON=python3
 ---> Using cache
 ---> d1a0d2565efb
Step 5/32 : ARG PIP=pip3
 ---> Using cache
 ---> 230ac67e8779
Step 6/32 : ARG TFS_SHORT_VERSION=2.0.1
 ---> Using cache
 ---> c597fd2bd758
Step 7/32 : ARG TFS_URL=https://tensorflow-aws.s3-us-west-2.amazonaws.com/${TFS_SHORT_VERSION}/Serving/CPU-WITH-MKL/tensorflow_model_server
 ---> Using cache
 ---> 4e394ef5ff7a
Step 8/32 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> 5d0c916d4c18
Step 9/32 : ENV PYTHONDONTWRITEBYTECODE=1
 ---> Using cache
 ---> 3731495125fa
Step 10/32 : ENV PYTHONUNBUFFERED=1
 ---> Using cache
 ---> 25e22cb6ec31
Step 11/32 : ENV SAGEMAKER_TFS_VERSION="${TF

## TF Serving:2.0.0-cpu 를 ECR에 퍼블리시

아래 셀 스크립트를 실행하면 위에서 생성한 이미지가 ECR에 퍼블리시 됩니다.
```
./scripts/publish.sh --version 1.13 --arch cpu
```

현재 (2020/07) ECR에 해당 이미지가 없으면  publish.sh 에서 에러가 발생하여 아래 코드 추가 함
```
aws ecr describe-repositories --repository-names $repository > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repository > /dev/null
fi
```


In [14]:
%%writefile sagemaker-tensorflow-serving-container/scripts/publish.sh

#!/bin/bash
#
# Publish images to your ECR account.

# 기존 소스에서 아래 주석 처리 함
# set -euo pipefail

source scripts/shared.sh

parse_std_args "$@"

aws ecr get-login-password --region ${aws_region} \
    | docker login \
        --password-stdin \
        --username AWS \
        "${aws_account}.dkr.ecr.${aws_region}.amazonaws.com/${repository}"

aws ecr describe-repositories --repository-names $repository > /dev/null 2>&1


if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repository > /dev/null
fi

docker tag $repository:$full_version-$device $aws_account.dkr.ecr.$aws_region.amazonaws.com/$repository:$full_version-$device
docker tag $repository:$full_version-$device $aws_account.dkr.ecr.$aws_region.amazonaws.com/$repository:$short_version-$device
docker push $aws_account.dkr.ecr.$aws_region.amazonaws.com/$repository:$full_version-$device
docker push $aws_account.dkr.ecr.$aws_region.amazonaws.com/$repository:$short_version-$device
docker logout https://$aws_account.dkr.ecr.$aws_region.amazonaws.com


Overwriting sagemaker-tensorflow-serving-container/scripts/publish.sh


In [15]:
%%bash -s {region} {tfs_version} {cpu_type}

AWS_DEFAULT_REGION=$1
export AWS_DEFAULT_REGION
TFS_VERION=$2
CPU_TYPE=$3

cd sagemaker-tensorflow-serving-container

./scripts/publish.sh --version $TFS_VERION --arch $CPU_TYPE


Login Succeeded
The push refers to repository [057716757052.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving]
27ad256900e2: Preparing
3a09520723c8: Preparing
af5b6438532e: Preparing
a0687c452f98: Preparing
48838a30321c: Preparing
e814f0abc0d3: Preparing
c128d5a19263: Preparing
b1f8aa56790d: Preparing
1066a9f9097f: Preparing
a7a05cc084a1: Preparing
20462666fdd8: Preparing
ee8a8520673c: Preparing
bd09e4e679fc: Preparing
8f4be5d31636: Preparing
001e4a80973b: Preparing
2ba5b91ca2b0: Preparing
2f37d1102187: Preparing
79bde4d54386: Preparing
ee8a8520673c: Waiting
c128d5a19263: Waiting
a7a05cc084a1: Waiting
bd09e4e679fc: Waiting
20462666fdd8: Waiting
b1f8aa56790d: Waiting
1066a9f9097f: Waiting
8f4be5d31636: Waiting
e814f0abc0d3: Waiting
001e4a80973b: Waiting
2ba5b91ca2b0: Waiting
79bde4d54386: Waiting
2f37d1102187: Waiting
27ad256900e2: Layer already exists
3a09520723c8: Layer already exists
af5b6438532e: Layer already exists
48838a30321c: Layer already exists
a0687c452f98: L

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



### Custom Inference Image 저장
<font color="red">아래의 그림처럼 계정 및 지역이 다르다면 수정해야 합니다.</font>

![ecr-infer-container](img/ecr-infer-container.png)

In [16]:

account = account_id
ecr_infer_custom_image_tf_serving_20_cpu = '{}.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:{}-{}'.format(
    account,
    tfs_version,
    cpu_type
)
print("ecr_infer_custom_image_tf_serving_20_cpu: \n", ecr_infer_custom_image_tf_serving_20_cpu)

ecr_infer_custom_image_tf_serving_20_cpu: 
 057716757052.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-tensorflow-serving:2.0.0-cpu


이미지를 다음 노트북에서 사용하기 위하여 저장 합니다.

In [17]:
%store ecr_infer_custom_image_tf_serving_20_cpu

Stored 'ecr_infer_custom_image_tf_serving_20_cpu' (str)
